In [1]:
########################################################################
### This program is a stock portfolio management tool.               ###
### It allows the user to manage stock portfolios.                   ###
### The program provides various functionalities such as retrieving  ###
### stock data, adding stocks to the portfolio, selling stocks,      ###
### and tracking portfolio performance.                              ###
########################################################################


# load modules
import yfinance as yf
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from urllib.error import HTTPError
from IPython.display import Markdown, display



# This function reads an existing portfolio
def load_existing_portfolio():
    global portfolio
    
    # read my_portfolio.csv file and return 1
    try:
        portfolio = pd.read_csv('my_portfolio.csv')
        return 1
    
    # If nonexistent, notify user and return 0
    except FileNotFoundError:
        print('\nFile not found. Please check the file name or path.')
        return 0
    
    
    
# This function creats a new portfolio
def create_new_portfolio():
    global portfolio
    portfolio = pd.DataFrame(columns=['Ticker', 'Average Purchase Price', 'Number of Stocks'])
    

    
# This function ensures that a valid stock ticker is entered
def get_right_ticker():
    
    # while true, keep asking for a ticker
    while True:
        
        # Get user input for the stock ticker
        ticker = str(input("Enter the stock ticker: ")).upper()
        try:
            
            # Fetch the stock information
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Check if the stock exists in Yahoo Finance
            if 'longName'not in info:
                
                # If not, raise NameError
                raise NameError()
                
            # Return ticker if successful
            return ticker
        
        # if an error occures, ask for another ticker
        except HTTPError:
            print("An HTTP error occurred, please try again or enter a different stock.")
        except NameError:
            print("A NameError occurred, please try again or enter a different stock.")
        except Exception:
            print("An error occured, please try again or enter a different stock.")        


    
# This function ensures that a valid number is entered 
def ask_for_number():
    while True:
        number = input("Enter number of stocks: ")
        
        # Valid number entered, return it
        try:
            number = float(number)
            return number
        
        # Invalid number entered, keep asking
        except ValueError:
            print("\nInvalid input. Please enter a valid number.")

            
# layout tool
def print_with_font_size(text, font_size):
    display(Markdown(f"<font size={font_size}>{text}</font>"))
    
    


# This function displays stock data for a specific stock
def get_stock_data():
    
    # Get user input for the stock ticker
    ticker = get_right_ticker()

    # Fetch the stock information
    stock = yf.Ticker(ticker)

    # Get the stock info
    info = stock.info

    # Font size 
    from IPython.display import Markdown, display

    # Print company name
    print_with_font_size(info.get('longName', 'N/A'), 5)

    # Print current market price
    print_with_font_size("Current Market Price: " + str(info.get('currentPrice', 'N/A')), 3)

    # GET TODAY'S DATE AND CONVERT IT TO A STRING WITH YYYY-MM-DD FORMAT (YFINANCE EXPECTS THAT FORMAT)# GET TODAY'S DATE AND CONVERT IT TO A STRING WITH YYYY-MM-DD FORMAT (YFINANCE EXPECTS THAT FORMAT)
    end_date = datetime.now().strftime('%Y-%m-%d')

    # GET HISTORICAL DATA FOR THE CHOSEN STOCK
    stock_hist = stock.history(start='2021-01-01', end=end_date)

    # PLOT THE ADJUSTED CLOSE PRICE
    plt.figure(figsize=(10, 4))
    plt.plot(stock_hist.index, stock_hist['Close'])
    plt.xlabel('Date')
    plt.ylabel('Adjusted Close Price')
    plt.title(f'Historical Stock Price for {ticker}', fontsize=14)
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

    # Print the financial information
    print("Overview of " + ticker + ": Stock Information, Financials, and Market Performance:\n")
    print("Company:                         {:<30}".format(info.get('longName', 'N/A')))
    print("Symbol:                          {:<30}".format(info.get('symbol', 'N/A')))
    print("Exchange:                        {:<30}".format(info.get('exchange', 'N/A')))
    print("Currency:                        {:<30}".format(info.get('currency', 'N/A')))
    print("Sector:                          {:<30}".format(info.get('sector', 'N/A')))
    print("Industry:                        {:<30}".format(info.get('industry', 'N/A')))
    print("Country:                         {:<30}".format(info.get('country', 'N/A')))
    print("Market Cap:                      {:<30}".format(info.get('marketCap', 'N/A')))
    print("Forward PE:                      {:<30}".format(info.get('forwardPE', 'N/A')))
    print("Trailing PE:                     {:<30}".format(info.get('trailingPE', 'N/A')))
    print("Price to Sales:                  {:<30}".format(info.get('priceToSalesTrailing12Months', 'N/A')))
    print("Dividend Rate:                   {:<30}".format(info.get('dividendRate', 'N/A')))
    print("Trailing Annual Dividend Yield:  {:<30}".format(info.get('trailingAnnualDividendYield', 'N/A')))
    print("Trailing Annual Dividend Rate:   {:<30}".format(info.get('trailingAnnualDividendRate', 'N/A')))
    print("Earnings Quarterly Growth:       {:<30}".format(info.get('earningsQuarterlyGrowth', 'N/A')))
    print("Revenue Growth:                  {:<30}".format(info.get('revenueGrowth', 'N/A')))
    print('\n')

    
    
# This function adds a specific stock to the portfolio
def add_stock_to_portfolio():
    
    global portfolio
    
    # get inputs: ticker, number of stocks, and purchase date
    ticker = get_right_ticker()
    num_stocks = ask_for_number()
    purchase_date = input("Enter date when bought (YYYY-MM-DD): ")
    
    # import purchase price
    purchase_price = yf.Ticker(ticker).history(start='1900-01-01', end=purchase_date)['Close'].values[-1]
    
    # check whether stock is in portfolio
    if ticker in portfolio['Ticker'].values:
        
        # calculate new av. purchase price and add new stocks
        old_num_stocks = portfolio.loc[portfolio['Ticker'] == ticker, 'Number of Stocks'].values[0]
        old_avg_price = portfolio.loc[portfolio['Ticker'] == ticker, 'Average Purchase Price'].values[0]
        new_avg_price = ((old_avg_price * old_num_stocks) + (purchase_price * num_stocks)) / (old_num_stocks + num_stocks)
        new_num_stocks = old_num_stocks + num_stocks
        portfolio.loc[portfolio['Ticker'] == ticker, 'Average Purchase Price'] = new_avg_price
        portfolio.loc[portfolio['Ticker'] == ticker, 'Number of Stocks'] = new_num_stocks
        
        # print result/summary
        print(f'\nYou bought {num_stocks} stocks of {ticker} at a price per share of ${purchase_price:.2f}.\nThis results in a transaction value of ${(purchase_price*num_stocks):.2f}.\nThe total number of {ticker} stocks in your portfolio is {new_num_stocks} with an average purchase price of ${new_avg_price:.2f}.\n')
    
    # else, add new line to portfolio
    else:
        portfolio = portfolio.append({'Ticker': ticker, 'Average Purchase Price': purchase_price, 'Number of Stocks': num_stocks}, ignore_index=True)
        print(f'\n{num_stocks} stocks of {ticker} at a price per share of ${purchase_price:.2f}.\nThis results in a transaction value of ${(purchase_price*num_stocks):.2f}.\n')

        

# This function removes stocks from the portfolio
def sell_stock():
    
    # get inputs: ticker, number of stocks, and sell date
    ticker = get_right_ticker()
    num_stocks_sold = ask_for_number()
    sell_date = input("Enter date when sold (YYYY-MM-DD): ")
    
    # import sell price
    sell_price = yf.Ticker(ticker).history(start='1900-01-01', end=sell_date)['Close'].values[-1]
    
    # check whether stock in portfolio
    if ticker not in portfolio['Ticker'].values:
        print("Stock does not exist in the portfolio.")
        return
    
    # check whether number of stocks sold is smaller than stocks in portfolio
    num_stocks = portfolio.loc[portfolio['Ticker'] == ticker, 'Number of Stocks'].values[0]
    if num_stocks < num_stocks_sold:
        print("You can only sell as many stocks as you have.")
        return
    
    # calculate proceeds and gains
    proceeds = sell_price * num_stocks_sold
    invested_capital = portfolio.loc[portfolio['Ticker'] == ticker, 'Average Purchase Price'].values[0] * num_stocks_sold
    multiple_of_invested_capital = proceeds / invested_capital
    gain = (multiple_of_invested_capital-1)*100
    
    # update number of stocks in portfolio
    new_num_stocks = num_stocks - num_stocks_sold
    portfolio.loc[portfolio['Ticker'] == ticker, 'Number of Stocks'] = new_num_stocks
    
    # print results/summary
    print(f'\nYou sold {num_stocks_sold} stocks of {ticker} at a price of ${sell_price:.2f}.')
    print(f'Proceeds: ${proceeds:.2f}')
    print(f'Gain/Loss: {gain:.2f}%\n')    
    print(f'Amount of {ticker} stocks left in portfolio: {new_num_stocks}\n')


    
    
# This function tracks the portfolio performance
def track_portfolio_performance():
    
    print_with_font_size(f'Your portfolio consists of the following stocks:\n', 3)
    
    total_invested_capital = 0
    total_current_value = 0
    
    # iterate through stocks in portfolio
    for index, row in portfolio.iterrows():
        
        # get ticket, number of stocks, and av. purchase price
        ticker = row['Ticker']
        num_stocks = row['Number of Stocks']
        avg_purchase_price = row['Average Purchase Price']
        
        # calculate gains
        current_price = yf.Ticker(ticker).history().tail(1)['Close'].values[0]
        invested_capital = avg_purchase_price * num_stocks
        current_value = current_price * num_stocks
        multiple_of_invested_capital = current_value / invested_capital
        gain = (multiple_of_invested_capital-1)*100
        
        # print results/summary
        print("Ticker:\t\t\t", ticker)
        print("Number of Shares:\t", num_stocks)
        print(f'Average Purchase Price:\t ${avg_purchase_price:.2f}')
        print(f'Current Price:\t\t ${current_price:.2f}')
        print(f'Gain/Loss:\t\t {gain:.2f}%\n')
        
        # raise total_invested_capital and total_current_value
        total_invested_capital += invested_capital
        total_current_value += current_value
    
    # calculate total gain
    total_multiple_of_invested_capital = total_current_value / total_invested_capital
    gain = (total_multiple_of_invested_capital-1)*100
    
    # print total summary
    print(f'\nTotal Portfolio Performance:')
    print(f'Invested Capital:\t ${total_invested_capital:.2f}')
    print(f'Portfolio Value:\t ${total_current_value:.2f}')
    print(f'Gain/Loss:\t\t {gain:.2f}%\n')  

    


### Main block
print('Welcome.')

# Ask user to choose between Existing Portfolio or New Portfolio
while True:
    print('\nWould you like to load an existing portfolio or create a new one?')
    print('1. Existing Portfolio')
    print('2. New Portfolio')
    
    # ask for input
    choice = input("Enter your choice: ")

    # call function 1 for existing portfolio
    if choice == '1':
        return_value = load_existing_portfolio()
        
        # only if return value 1, break. Otherwise ask again.
        if return_value == 1:
            break
        else:
            continue
    
    # call function 2 for new portfolio
    elif choice == '2':
        create_new_portfolio()
        break
        
    else:
        print("Invalid choice. Please try again.")



# Ask user to choose between 5 actions
while True:
    print("\nSelect an option:")
    print("1. Get Stock Data")
    print("2. Add Stock to Portfolio")
    print("3. Sell Stock")
    print("4. Track Portfolio Performance")
    print("5. Exit")
    
    # ask for input
    choice = input("Enter your choice: ")
    
    # call respective function
    if choice == '1':
        get_stock_data()   
    elif choice == '2':
        add_stock_to_portfolio()
    elif choice == '3':
        sell_stock()
    elif choice == '4':
        track_portfolio_performance()
    elif choice == '5':

        # export portolio to current working directory
        portfolio.to_csv('my_portfolio.csv', index=True)
        print('Your portfolio is saved as "my_portfolio.csv" in your current working directory.\nGoodbye.')
        break
    
    # invalid choice: try again
    else:
        print("Invalid choice. Please try again.")



Welcome.

Would you like to load an existing portfolio or create a new one?
1. Existing Portfolio
2. New Portfolio
Enter your choice: 1

Select an option:
1. Get Stock Data
2. Add Stock to Portfolio
3. Sell Stock
4. Track Portfolio Performance
5. Exit
Enter your choice: 5
Your portfolio is saved as "my_portfolio.csv" in your current working directory.
Goodbye.
